In [1]:
!pip install polars openpyxl xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.3 MB/s eta 0:00:0000:01


In [ ]:
import polars as pl
i = '2025-11-10'

print(pl.date(2025,11,10))

2025-11-10 00:00:00.alias("datetime").strict_cast(Date).alias("date")
2025-11-10 00:00:00.alias("datetime").strict_cast(Date).alias("date")


In [2]:
import polars as pl
import pandas as pd 
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

account_name = "ststbrawadjrsbxsea001"
account_key = "EvVJvH/9JGncF743x9hLYWWedJCjbch8UGm81uu5tkEpNOoi8qc3Q+QvisTTg1aTZh1c3FCBvS5W+AStjHx5tg=="
storage_options = {
    "account_name": account_name,
    "account_key":  account_key,
}


# Tổng hợp Stats

In [ ]:
start_date = '2025-07-22'
end_date = '2025-11-28'
need_call = pd.date_range(start_date, end_date).strftime("%Y-%m-%d").tolist()
from datetime import datetime, timezone, timedelta
timezone_offset = 7.0 
tzinfo = timezone(timedelta(hours=timezone_offset))
now = datetime.now(tz=tzinfo).strftime("%Y%m%dT%H%M%S")
date = datetime.now(tz=tzinfo).strftime("%Y%m%d")
print((now,date))
import os
os.makedirs(f'AdhocOutput/{date}',exist_ok=True)

from tqdm import tqdm 
import re
dfs = []
for i in tqdm(need_call):
# for i in ['2025-10-10','2025-10-11']:
    df = pl.scan_parquet(f"az://adjuststbuatprocessed/live/output/{i}.parquet",
                         storage_options=storage_options,
                         glob = True)

    colnames = [re.findall(r'(?<=\{)[^}]+(?=\})', i)[0] for i in list(df.collect_schema())[:-1]] + [list(df.collect_schema())[-1]]
    col_dic = {key:val for key, val in zip(list(df.collect_schema()), colnames)}
    df = df.rename(col_dic)
    df = df.with_columns(
        (pl.from_epoch('created_at', time_unit='s') + pl.duration(hours = 7)).alias('created_at_convert'),
        (pl.from_epoch('installed_at', time_unit='s') + pl.duration(hours = 7)).alias('installed_at_convert'),
        (pl.from_epoch('engagement_time', time_unit='s') + pl.duration(hours = 7)).alias('engagement_time_convert')
    ).with_columns(
        pl.col('created_at_convert').dt.date().alias('created_at_convert_date'),
        pl.col('installed_at_convert').dt.date().alias('installed_at_convert_date'),
        pl.col('engagement_time_convert').dt.date().alias('engagement_time_convert_date'),
        cif=(
            pl.when(
                pl.col("partner_parameters").is_not_null()
                | (pl.col("partner_parameters") != "")
            )
            .then(
                pl.coalesce(pl.col("partner_parameters").str.json_path_match("$.userid"),
                            pl.col("partner_parameters").str.json_path_match("$.userId"),
                            pl.col("partner_parameters").str.json_path_match("$.userid:"),
                            pl.col("partner_parameters").str.json_path_match("$.userId:"))  # → "20516726"
            )
            .otherwise(None)
        )
    )
    _ = df.filter(
        pl.col.activity_kind == 'install',
        pl.col.created_at_convert_date == (pl.date(int(i[:4]),int(i[5:7]),int(i[8:10])))
    ).group_by(['created_at_convert_date','network_name','campaign_name']).agg(
        pl.len().alias("install_num")
    )
    __ = df.filter(
        pl.col.event_name == 'open_account_complete',
        pl.col.created_at_convert_date == (pl.date(int(i[:4]),int(i[5:7]),int(i[8:10])))
    ).group_by(['created_at_convert_date','network_name','campaign_name']).agg(
        pl.col.adid.n_unique().alias("open_acc_complete_num_uniq")
    )
    ___ = df.filter(
        pl.col.activity_kind == 'click',
        pl.col.created_at_convert_date == (pl.date(int(i[:4]),int(i[5:7]),int(i[8:10])))
    ).group_by(['created_at_convert_date','network_name','campaign_name']).agg(
        pl.col.adid.n_unique().alias("click_num_uniq")
    )
    final = _.join(__, how = 'left', on = ['created_at_convert_date','network_name','campaign_name'])
    final = final.join(___, how = 'left', on = ['created_at_convert_date','network_name','campaign_name']).with_columns(
        pl.col.open_acc_complete_num_uniq.fill_null(strategy="zero"),
        pl.col.click_num_uniq.fill_null(strategy="zero")).collect().to_pandas()
    dfs.append(final)
    # if i == '2025-07-25': break 
data_frame = pd.concat(dfs, axis = 0, ignore_index=True)
data_frame.tail()
# data_frame.to_excel(f"AdhocOutput/{date}/Prod_AT-Oct_{now}.xlsx", index=False)

('20251128T155745', '20251128')


100%|██████████| 130/130 [17:25<00:00,  8.05s/it]


,created_at_convert_date,network_name,campaign_name,install_num,open_acc_complete_num_uniq,click_num_uniq
3672,2025-11-28,Google Organic Search,sacombank pay,1,0,0
3673,2025-11-28,CTKM,Mở TKTT,24,8,4
3674,2025-11-28,AccessTrade Affiliate,AT_2087468,1,0,1
3675,2025-11-28,Google Organic Search,tài app sacombank pay,1,0,0
3676,2025-11-28,Google Organic Search,tải app sacombank pay,1,0,0


In [ ]:
data_frame.to_excel(f"AdhocOutput/{date}/Prod_AT-Oct_{now}.xlsx", index=False)

In [ ]:
start_date = '2025-07-22'
end_date = '2025-11-28'
need_call = pd.date_range(start_date, end_date).strftime("%Y-%m-%d").tolist()
from datetime import datetime, timezone, timedelta
timezone_offset = 7.0 
tzinfo = timezone(timedelta(hours=timezone_offset))
now = datetime.now(tz=tzinfo).strftime("%Y%m%dT%H%M%S")
date = datetime.now(tz=tzinfo).strftime("%Y%m%d")
print((now,date))
import os
os.makedirs(f'AdhocOutput/{date}',exist_ok=True)

from tqdm import tqdm 
import re
dfs = []
for i in tqdm(need_call):
# for i in ['2025-10-10','2025-10-11']:
    df = pl.scan_parquet(f"az://adjuststbuatprocessed/live/output/{i}.parquet",
                         storage_options=storage_options,
                         glob = True)

    colnames = [re.findall(r'(?<=\{)[^}]+(?=\})', i)[0] for i in list(df.collect_schema())[:-1]] + [list(df.collect_schema())[-1]]
    col_dic = {key:val for key, val in zip(list(df.collect_schema()), colnames)}
    df = df.rename(col_dic)
    df = df.with_columns(
        (pl.from_epoch('created_at', time_unit='s') + pl.duration(hours = 7)).alias('created_at_convert'),
        (pl.from_epoch('installed_at', time_unit='s') + pl.duration(hours = 7)).alias('installed_at_convert'),
        (pl.from_epoch('engagement_time', time_unit='s') + pl.duration(hours = 7)).alias('engagement_time_convert')
    ).with_columns(
        pl.col('created_at_convert').dt.date().alias('created_at_convert_date'),
        pl.col('installed_at_convert').dt.date().alias('installed_at_convert_date'),
        pl.col('engagement_time_convert').dt.date().alias('engagement_time_convert_date'),
        cif=(
            pl.when(
                pl.col("partner_parameters").is_not_null()
                | (pl.col("partner_parameters") != "")
            )
            .then(
                pl.coalesce(pl.col("partner_parameters").str.json_path_match("$.userid"),
                            pl.col("partner_parameters").str.json_path_match("$.userId"),
                            pl.col("partner_parameters").str.json_path_match("$.userid:"),
                            pl.col("partner_parameters").str.json_path_match("$.userId:"))  # → "20516726"
            )
            .otherwise(None)
        )
    )
    _a = df.with_columns(pl.coalesce(pl.col.event_name, pl.col.activity_kind).alias('event')).filter(
        (pl.col.event.is_in(['install','open_account_complete'])),
        # pl.col.created_at_convert_date == (pl.date(int(i[:4]),int(i[5:7]),int(i[8:10])))
    ).select(['created_at_convert','network_name','campaign_name', 'adid','event']).collect().to_pandas()
    
    dfs.append(_a)
    # if i == '2025-07-25': break 
data_frame_2 = pd.concat(dfs, axis = 0, ignore_index=True)
# data_frame_2
# data_frame.to_excel(f"AdhocOutput/{date}/Prod_AT-Oct_{now}.xlsx", index=False)

('20251128T163218', '20251128')


  2%|▏         | 3/130 [00:02<01:57,  1.08it/s]


,created_at_convert,network_name,campaign_name,adid,event
0,2025-07-22 15:30:39,Organic,None,40d73e215de75e911a90447fb4bb25a0,install
1,2025-07-22 15:43:53,SACOMBANKPAY PILOT,Mở tài khoản thanh toán,ce552fe37c860e53336a07d3a55e5af8,install
2,2025-07-22 16:15:43,Organic,None,2780ce44d3778ccff15f2aac40761b18,install
3,2025-07-22 16:32:45,SACOMBANKPAY PILOT,Mua ngoại tệ,39655811ccadaf2169a4aea1724b26d0,install
4,2025-07-22 16:03:00,SACOMBANKPAY PILOT,Mua ngoại tệ,8cb9de197010fc63b2ebdc17acde0e52,install
5,2025-07-22 16:24:20,Organic,None,c6cf20e2a32ffde68ec915fbee21c6b9,install
6,2025-07-22 16:47:33,SACOMBANKPAY PILOT,Vay tiêu dùng,7ca39af3788931b15164c705c471b81b,install
7,2025-07-22 16:33:56,Organic,None,26eef68f9dd4010a64005db22e059fae,install
8,2025-07-22 16:36:53,SACOMBANKPAY PILOT,Mua ngoại tệ,7ca39af3788931b15164c705c471b81b,install
9,2025-07-22 17:29:41,Organic,None,0c6b0e3f05634e7720b42eb1ebe6721d,install


In [6]:
paths = [f'AdhocOutput/{date}/prod_AT-{i}_{now}.csv' for i in need_call]
data_frame = pd.concat([pd.read_csv(path) for path in paths], axis = 0, ignore_index=True)
data_frame.head()

/tmp/ipykernel_2164/3636695480.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_frame = pd.concat([pd.read_csv(path) for path in paths], axis = 0, ignore_index=True)


,adid,app_version_short,network_name,campaign_name,activity_kind,event_name,created_at_convert,created_at_convert_date,partner_parameters,cif,publisher_parameters,label,first_tracker,tracker_name,adgroup_name,creative_name,reftag,device_manufacturer,city,country,country_subdivision,device_model,device_name,device_type,environment,os_name,os_version,installed_at_convert,installed_at_convert_date,idfa,idfv,gps_adid
0,114c85e22ff0b1a69aa5ab1a9f822551,2.6.1,AccessTrade Affiliate,AT_2096538,event,open_account_complete,2025-10-08 16:20:03,2025-10-08,"{""userId"":""20414173""}",20414173.0,NaN,NaN,1t7xzk1v,AccessTrade Affiliate::AT_2096538::AT_2096538_...,AT_2096538_AJ1_ANG,NaN,cf1bUE8DHfsN8,Tecno,Da Nang,vn,Da Nang,NaN,Spark10Pro,phone,production,android,13,2025-10-08 15:35:57,2025-10-08,NaN,NaN,b814c930-c228-4fc2-91c1-89c483dd4762
1,01dd2c474767957fb549e535c622cdab,2.6.1,AccessTrade Affiliate,AT_2065891,event,open_account_complete,2025-10-10 03:14:21,2025-10-10,NaN,NaN,NaN,NaN,1ttmmc2v,AccessTrade Affiliate::AT_2065891::AT_2065891_...,AT_2065891_BLND_AH,NaN,cz7PWHK7iz6XK,Samsung,Hanoi,vn,Hanoi,NaN,GalaxyS225G,phone,production,android,14,2025-10-10 03:03:04,2025-10-10,NaN,NaN,d6668e0a-2abe-4ecf-aed7-fd708d413bcb
2,02d462dc52fd29dd85d735f1bf246088,2.6.1,AccessTrade Affiliate,AT_2087468,event,open_account_complete,2025-10-09 22:47:28,2025-10-09,NaN,NaN,NaN,NaN,1txnjeez,AccessTrade Affiliate::AT_2087468::AT_2087468_...,AT_2087468_IGK_40307956HUB,NaN,cYDZUH4EEcctW,Google,Hưng Yên,vn,Bình Phước Province,NaN,Pixel7,phone,production,android,14,2025-10-09 22:36:19,2025-10-09,NaN,NaN,c6bc813c-b540-4fe9-8be6-eace35637104
3,037fc82f74e9a74f2efc99d1f0fd14f1,2.6.1,AccessTrade Affiliate,AT_2065891,event,open_account_complete,2025-10-10 04:05:06,2025-10-10,NaN,NaN,NaN,NaN,1ttmmc2v,AccessTrade Affiliate::AT_2065891::AT_2065891_...,AT_2065891_BLND_AH,NaN,c6PBSy5mjwhOZ,HMD,Hanoi,vn,Hanoi,NaN,NokiaC31,phone,production,android,12,2025-10-10 03:55:15,2025-10-10,NaN,NaN,c505f111-68cb-4ad2-939d-3c1e39ce1d64
4,038dfab829e89724297c5bb4917d3f02,2.6.1,AccessTrade Affiliate,AT_2087468,event,open_account_complete,2025-10-09 21:25:10,2025-10-09,NaN,NaN,NaN,NaN,1ta2mt2u,AccessTrade Affiliate::AT_2087468::AT_2087468_...,AT_2087468_IGK_40307956gQ,NaN,cCw3qNuvzYWMj,Samsung,Haiphong,vn,Haiphong,NaN,GalaxyA21s,phone,production,android,12,2025-10-09 21:13:30,2025-10-09,NaN,NaN,1cef8548-703b-4707-adb7-9c4fd80e0682


# Lấy CIF

In [36]:
# need_call = ['2025-11-19','2025-11-20','2025-11-21','2025-11-22']
df = pl.scan_parquet(
    # "az://adjuststbuatprocessed/output/2025-11-21.parquet",
    "az://adjuststbuatprocessed/live/output/2025-10-17.parquet",
    # "az://adjuststbuatprocessed/live/processing/dt=2025-10-07/*.parquet",
    # [f"az://adjuststbuatprocessed/output/{i}.parquet" for i in need_call],
    # [f"az://adjuststbuatprocessed/live/output/{i}.parquet" for i in need_call],
    storage_options=storage_options,
    glob=True,                 # rất quan trọng để expand '*'
)


In [37]:
import re
colnames = [re.findall(r'(?<=\{)[^}]+(?=\})', i)[0] for i in list(df.collect_schema())[:-1]] + [list(df.collect_schema())[-1]]
col_dic = {key:val for key, val in zip(list(df.collect_schema()), colnames)}
df = df.rename(col_dic)
df = df.with_columns(
    (pl.from_epoch('created_at', time_unit='s') + pl.duration(hours = 7)).alias('created_at_convert'),
    (pl.from_epoch('installed_at', time_unit='s') + pl.duration(hours = 7)).alias('installed_at_convert'),
    (pl.from_epoch('engagement_time', time_unit='s') + pl.duration(hours = 7)).alias('engagement_time_convert'),
    (pl.from_epoch('click_time', time_unit='s') + pl.duration(hours = 7)).alias('click_time_convert')
).with_columns(
    pl.col('created_at_convert').dt.date().alias('created_at_convert_date'),
    pl.col('installed_at_convert').dt.date().alias('installed_at_convert_date'),
    pl.col('engagement_time_convert').dt.date().alias('engagement_time_convert_date'),
    pl.col('click_time_convert').dt.date().alias('click_time_convert_date'),
    cif=(
        pl.when(
            pl.col("partner_parameters").is_not_null()
            | (pl.col("partner_parameters") != "")
        )
        .then(
            pl.coalesce(
                pl.col("partner_parameters")
            .str.extract(r'(?i)"userid"\s*:\s*"([^"]+)"', 1),  # → "20516726"
                pl.col("partner_parameters")
            .str.extract(r'(?i)"userid:"\s*:\s*"([^"]+)"', 1),
                pl.col("partner_parameters")
            .str.extract(r'(?i)"userId"\s*:\s*"([^"]+)"', 1),  # → "20516726"
                pl.col("partner_parameters")
            .str.extract(r'(?i)"userId:"\s*:\s*"([^"]+)"', 1)
            )
        )
        .otherwise(None)
        )
    )
    

In [38]:
selected = ['adid','app_version_short','network_name','campaign_name','activity_kind', 'event_name', 'created_at_convert','created_at_convert_date','partner_parameters','cif','publisher_parameters','label', 'first_tracker','tracker_name', 'adgroup_name', 'creative_name']
device_cols = [
    'reftag','reftags',
    # 'app_version_short',
    'device_manufacturer',
    'city','country','country_subdivision',
    'device_model','device_name','device_type','environment',
    'os_name','os_version', 'installed_at_convert', 'installed_at_convert_date',
    'idfa','idfv', 'gps_adid'
]
cols = selected + device_cols


In [27]:
df.select(pl.col.created_at_convert.max()).collect()

created_at_convert
datetime[μs]
2025-12-03 06:59:30


In [39]:
cif = '20434448'
adids = df.filter(
    (pl.col.cif == cif) | (pl.col.partner_parameters.str.contains(cif))
).select(pl.col.adid.unique()).collect(engine = 'streaming').to_pandas().adid.to_list()
print(adids)

['4e7802242b10c612d79c134266aadca0']


In [40]:
reftags = df.filter(
    pl.col.adid.is_in(adids),
    pl.col.activity_kind == 'install'
).select(pl.col.reftags.unique()).collect(engine = 'streaming').to_pandas().reftags.to_list()
print(reftags)

['["cb4zsstbPo2Cy"]']


In [41]:
reftags = eval(reftags[0])
reftags

['cb4zsstbPo2Cy']

In [ ]:
from google.colab import files
files.download('your_generated_file.csv')

In [42]:
info_ = df.filter(
    (pl.col.adid.is_in(adids))| (pl.col.reftag.is_in(reftags)) #| (pl.col.reftag == 'oPeMJb4H4fEiy')
).select(cols).sort(['adid','created_at_convert']).collect(engine = 'streaming').to_pandas()
info_ 

,adid,app_version_short,network_name,campaign_name,activity_kind,event_name,created_at_convert,created_at_convert_date,partner_parameters,cif,publisher_parameters,label,first_tracker,tracker_name,adgroup_name,creative_name,reftag,reftags,device_manufacturer,city,country,country_subdivision,device_model,device_name,device_type,environment,os_name,os_version,installed_at_convert,installed_at_convert_date,idfa,idfv,gps_adid
0,None,None,CTKM,Mở TKTT,click,None,2025-10-17 08:39:46,2025-10-17,None,None,None,STB031657,None,CTKM::Mở TKTT,None,None,cb4zsstbPo2Cy,None,None,Ho Chi Minh City,vn,Ho Chi Minh,None,GalaxyA03s,phone,None,android,13,NaT,NaT,None,None,None
1,4e7802242b10c612d79c134266aadca0,2.6.1,CTKM,Mở TKTT,event,open_app,2025-10-17 08:42:46,2025-10-17,None,None,None,STB031657,1two7d0d,CTKM::Mở TKTT,None,None,cb4zsstbPo2Cy,None,Samsung,Ho Chi Minh City,vn,Ho Chi Minh,None,GalaxyA03s,phone,production,android,13,2025-10-17 08:42:46,2025-10-17,None,None,c9086725-cf17-48f9-a618-4fd62a4315fe
2,4e7802242b10c612d79c134266aadca0,2.6.1,CTKM,Mở TKTT,install,None,2025-10-17 08:42:46,2025-10-17,None,None,None,STB031657,1two7d0d,CTKM::Mở TKTT,None,None,cb4zsstbPo2Cy,"[""cb4zsstbPo2Cy""]",Samsung,Ho Chi Minh City,vn,Ho Chi Minh,None,GalaxyA03s,phone,production,android,13,2025-10-17 08:42:46,2025-10-17,None,None,c9086725-cf17-48f9-a618-4fd62a4315fe
3,4e7802242b10c612d79c134266aadca0,2.6.1,CTKM,Mở TKTT,event,ekyc_start,2025-10-17 08:43:10,2025-10-17,None,None,None,STB031657,1two7d0d,CTKM::Mở TKTT,None,None,cb4zsstbPo2Cy,None,Samsung,Ho Chi Minh City,vn,Ho Chi Minh,None,GalaxyA03s,phone,production,android,13,2025-10-17 08:42:46,2025-10-17,None,None,c9086725-cf17-48f9-a618-4fd62a4315fe
4,4e7802242b10c612d79c134266aadca0,2.6.1,CTKM,Mở TKTT,event,first_login,2025-10-17 08:53:25,2025-10-17,"{""userId"":""20434448""}",20434448,None,STB031657,1two7d0d,CTKM::Mở TKTT,None,None,cb4zsstbPo2Cy,None,Samsung,Ho Chi Minh City,vn,Ho Chi Minh,None,GalaxyA03s,phone,production,android,13,2025-10-17 08:42:46,2025-10-17,None,None,c9086725-cf17-48f9-a618-4fd62a4315fe
5,4e7802242b10c612d79c134266aadca0,2.6.1,CTKM,Mở TKTT,event,open_app,2025-10-17 08:58:50,2025-10-17,None,None,None,STB031657,1two7d0d,CTKM::Mở TKTT,None,None,cb4zsstbPo2Cy,None,Samsung,Ho Chi Minh City,vn,Ho Chi Minh,None,GalaxyA03s,phone,production,android,13,2025-10-17 08:42:46,2025-10-17,None,None,c9086725-cf17-48f9-a618-4fd62a4315fe
6,4e7802242b10c612d79c134266aadca0,2.6.1,CTKM,Mở TKTT,event,open_app,2025-10-17 08:59:54,2025-10-17,None,None,None,STB031657,1two7d0d,CTKM::Mở TKTT,None,None,cb4zsstbPo2Cy,None,Samsung,Ho Chi Minh City,vn,Ho Chi Minh,None,GalaxyA03s,phone,production,android,13,2025-10-17 08:42:46,2025-10-17,None,None,c9086725-cf17-48f9-a618-4fd62a4315fe
7,4e7802242b10c612d79c134266aadca0,2.6.1,CTKM,Mở TKTT,event,open_app,2025-10-17 09:00:25,2025-10-17,None,None,None,STB031657,1two7d0d,CTKM::Mở TKTT,None,None,cb4zsstbPo2Cy,None,Samsung,Ho Chi Minh City,vn,Ho Chi Minh,None,GalaxyA03s,phone,production,android,13,2025-10-17 08:42:46,2025-10-17,None,None,c9086725-cf17-48f9-a618-4fd62a4315fe
8,4e7802242b10c612d79c134266aadca0,2.6.1,CTKM,Mở TKTT,event,open_app,2025-10-17 09:28:01,2025-10-17,None,None,None,STB031657,1two7d0d,CTKM::Mở TKTT,None,None,cb4zsstbPo2Cy,None,Samsung,Ho Chi Minh City,vn,Ho Chi Minh,None,GalaxyA03s,phone,production,android,13,2025-10-17 08:42:46,2025-10-17,None,None,c9086725-cf17-48f9-a618-4fd62a4315fe
9,4e7802242b10c612d79c134266aadca0,2.6.1,CTKM,Mở TKTT,event,open_app,2025-10-17 11:35:24,2025-10-17,None,None,None,STB031657,1two7d0d,CTKM::Mở TKTT,None,None,cb4zsstbPo2Cy,None,Samsung,Ho Chi Minh City,vn,Ho Chi Minh,None,GalaxyA03s,phone,production,android,13,2025-10-17 08:42:46,2025-10-17,None,None,c9086725-cf17-48f9-a618-4fd62a4315fe


In [25]:
from datetime import datetime, timezone, timedelta
timezone_offset = 7.0 
tzinfo = timezone(timedelta(hours=timezone_offset))
# datetime.now(tzinfo)
now = datetime.now(tz=tzinfo).strftime("%Y%m%dT%H%M%S")
date = datetime.now(tz=tzinfo).strftime("%Y%m%d")
print((now,date))
import os
os.makedirs(f'AdhocOutput/{date}',exist_ok=True)
df.select(cols).filter(
    pl.col.adid.is_in(adids), pl.col.created_at_convert_date.is_between(pl.date(2025,11,20),pl.date(2025,11,22))
    ).sort(['adid','created_at_convert']).collect().to_pandas().to_csv(f'AdhocOutput/{date}/prod_{'-'.join(need_call)}_{now}.csv', index = False)

('20251124T143910', '20251124')


In [ ]:
# cifs = ['19007449']
# adids = df.filter(
#     pl.col.partner_parameters.str.contains_any(cifs) | pl.col.label.str.contains("0886962308")
# ).select(pl.col.adid.unique()).collect(engine = 'streaming').to_pandas().adid.to_list()
# print(adids)

['d4b21bd40b091f1e2293b6c5c1aedf82']


In [ ]:
date = "20251121"
import os
os.makedirs(f'AdhocOutput/{date}',exist_ok=True)
df.select(cols).filter(pl.col.adid.is_in(adids)).sort(['adid','created_at_convert']).collect().to_pandas().to_csv(f'AdhocOutput/{date}/uat_{'-'.join(cifs)}_{date}T1600.csv', index = False)

In [9]:
adids = df.filter(
    pl.col.created_at_convert.is_between(pl.datetime(2025,11,18,10,41),pl.datetime(2025,11,18,10,43)),
    pl.col.event_name.is_in(["ekyc_success",'open_account_complete'])
          ).select(pl.col.adid.unique()).collect(engine = 'streaming').to_pandas().adid.to_list()
print(adids)

['a4393d2e7ca86ea24f871a470085e3b1', 'd11c8e99782ea1c789c2e685acb0b99b', 'c9b0dedeadd76975e1b4d33ef857c144', '7948a5db70e10e3f1aa2cb3eb0c8c671', '77e5980762c2c17fc673781f0e2412fb', '09d394832cba44b3344c6479fa0bac83', '4becb4009c097b298117c86da9d9caa3']


In [10]:
df.select(cols).filter(pl.col.adid.is_in(adids)).sort(['adid','created_at_convert']).collect().to_pandas().to_csv(f'prod_20251118T1041-43_20251120T1000.csv', index = False)